<a href="https://colab.research.google.com/github/Seongwoong-sk/PyTorch-Deep-Learning-Zero-To-All/blob/main/lab_11_5_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN - Seq2Seq
**Seq2Seq**    
- sequence를 입력받고 seq를 출력하는 모델
- [x] 일반 RNN과의 차이점
  - 챗봇 같은 상황에서 RNN는 한 단어를 받고 출력

**Apply Seq2Seq**    
- Encoder - Decoder
  - Encoder : 입력된 seq를 어떤 vector의 형태로 압축을 함 / 압축된 vector를 decoder에 전달
  - Decoder : encoder에서 만들어서 넘겨준 이 vector를 Decoder의 첫 셀의 Hidden State로 넣어주고 이 문장이 시작한다는 어떤 start flag와 함께 모델 시작
    - 이 셀에서 나온 ouput을 이 문장의 만들어진 답변의 첫 번째 단어로 두고 이 단어가 두 번째 셀의 입력에 들어가서 이 전 셀의 Hidden State와 함께 다음 단어를 예측
- Data Preprocessing
- Neural Net Setting
- Training
- Evaluation

In [ ]:
# main reference
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [ ]:
import random
import torch
import torch.nn as nn
from torch import optim

In [ ]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

In [ ]:
# fix token for "start of sentence" and "end of sentence"

# Start of Sentence
# >> decoder가 Hidden STate로 Encoder의 마지막 출력물을 받아오고 첫 번째 step의 input을 받아야 됨
# >> 그 step의 input으로 SOS_token을 넣어주게 됨
SOS_token = 0

# End of Sentence
# >> 문장이 끝나게 되면 Eos라는 토큰을 붙여서 문장의 종료를 알림
EOS_token = 1

In [ ]:
# class for vocabulary related information of data

class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>": SOS_token, "<EOS>": EOS_token}
        self.index2vocab = {SOS_token: "<SOS>", EOS_token: "<EOS>"}
        self.vocab_count = {}
        self.n_vocab = len(self.vocab2index)

    def add_vocab(self, sentence):
        for word in sentence.split(" "):
            if word not in self.vocab2index:
                self.vocab2index[word] = self.n_vocab
                self.vocab_count[word] = 1
                self.index2vocab[self.n_vocab] = word
                self.n_vocab += 1
            else:
                self.vocab_count[word] += 1

In [ ]:
# filter out the long sentence from source and target data

def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length

In [ ]:
# read and preprocess the corpus data

def preprocess(corpus, source_max_length, target_max_length):
    print("reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])
    print("Read {} sentence pairs".format(len(pairs)))

    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("Trimmed to {} sentence pairs".format(len(pairs)))

    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size =", source_vocab.n_vocab)
    print("target vocab size =", target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab

In [ ]:
# declare simple encoder

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size) # input : source corpus에서 사용되고 있는 단어 // 들어온 input을 가지고 hidden size만큼의 vector로 줄이는(훨씬 적은 차원을 가지는) 그런 vector로 표현되는 그런 matrix를 embedding이라고 함 
        self.gru = nn.GRU(hidden_size, hidden_size) # dimension of input vector, output vector

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        return x, hidden

In [ ]:
# declare simple decoder

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)
        x = self.softmax(self.out(x[0])) # gru를 거치고 나온 차원을 source의 단어 수에 맞게 변형하는 linear
        return x, hidden

In [ ]:
# convert sentence to the index tensor
# >> sentence를 one-hot encoding으로 바꾸고 그 one-hot vector를 이제 PyTorch에서 사용하는 tensor의 형태로 바꿔주는 과정

def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]
    indexes.append(vocab.vocab2index["<EOS>"])
    return torch.Tensor(indexes).long().to(device).view(-1, 1)

In [ ]:
# training seq2seq
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
    loss_total = 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

    criterion = nn.NLLLoss() # Negative Log Likelihood
                             # category value. 최종적으로 출력으로 나오는 이 단어(vector)들이 원래의 단어와 얼마나 잘 맞는지 category value들끼리 비교를 하게 될 때 얼마나 잘 맞는지 확인하는 loss 

    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1] # 입력은 최초 단어
        target_tensor = training_target[i - 1]

        # encoder의 첫 gru에 들어가는  Hidden State가 없어서
        # 0 vector로 다 만들어가지고 넣어줌. 그래서 첫 번째 gru 셀에서 연산이 이루어짐
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0

        for enc_input in range(source_length):
        
        # 이 문장이 다 끝날 때까지 이 loop를 돌면서 encoder에서 Hidden State를 꺼내오게 됨
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

        # encoder의 마지막 Hidden State를 decoder의 첫 HIdden State에 넣어주는 과정
        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden # connect encoder output to decoder input

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing : 이미 알고 있는 label 문장에서 넣어주기

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length
        loss_total += loss_iter

        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

In [ ]:
# insert given sentence to check the training
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print(">", pair[0])
        print("=", pair[1])
        source_tensor = tensorize(source_vocab, pair[0])
        source_length = source_tensor.size()[0]
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)

        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]], device=device).long()
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)
            if top_index.item() == EOS_token:
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("<", predict_sentence)
        print("")

번역 task 전체의 흐름

In [ ]:
# declare max length for sentence
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

In [ ]:
# preprocess the corpus

load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print(random.choice(load_pairs))

reading corpus...
Read 4 sentence pairs
Trimmed to 4 sentence pairs
Counting words...
source vocab size = 17
target vocab size = 13
['i feel hungry.', '나는 배가 고프다.']


In [ ]:
# declare the encoder and the decoder

enc_hidden_size = 16

dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

In [ ]:
# train seq2seq model

train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 5000, print_every=1000)

[1000 - 20.0%] loss = 0.7381
[2000 - 40.0%] loss = 0.1091
[3000 - 60.0%] loss = 0.0359
[4000 - 80.0%] loss = 0.0188
[5000 - 100.0%] loss = 0.0127


In [ ]:
# check the model with given data

evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

> i feel hungry.
= 나는 배가 고프다.
< 나는 배가 고프다. <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
< 파이토치는 매우 쉽다. <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
< 파이토치는 딥러닝을 위한 프레임워크이다. <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
< 파이토치는 사용하기 매우 직관적이다. <EOS>

